In [5]:
import pandas as pd
import sqlite3

In [56]:
connection = sqlite3.connect("olist.db")

df_reviews = pd.read_sql_query("SELECT * FROM Reviews",connection)

connection.close()

In [12]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98410 entries, 0 to 98409
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                98410 non-null  object
 1   order_id                 98410 non-null  object
 2   review_score             98410 non-null  int64 
 3   review_comment_title     98410 non-null  object
 4   review_comment_message   98410 non-null  object
 5   review_creation_date     98410 non-null  object
 6   review_answer_timestamp  98410 non-null  object
 7   timestamp_field_7        26 non-null     object
dtypes: int64(1), object(7)
memory usage: 6.0+ MB


In [15]:
df_reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,15197aa66ff4d0650b5434f1b46cda19,b18dcdf73be66366873cd26c5724d1dc,1,,,2018-04-13 00:00:00,2018-04-16T00:39:37Z
1,9a0abbb668bafb95a6d2b05db43284c4,d7bd0e4afdf94846eb73642b4e3e75c3,3,,,2017-04-30 00:00:00,2017-05-03T00:02:22Z
2,fdbdb2629a7cde0f66657acc92084e7f,70a752414a13d09cc1f2b437b914b28e,3,,,2017-09-29 00:00:00,2017-10-02T01:12:49Z
3,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,,,2018-01-18 00:00:00,2018-01-18T21:46:59Z
4,23f75a37effc35d9a915b4e1ad483793,2eaf8e099d871cd5c22b83b5ea8f6e0e,4,,,2018-03-28 00:00:00,2018-03-30T15:10:55Z


# 1. Nettoyage

## 1.1 Gestion des valeurs manquantes

In [57]:
df_reviews = df_reviews.drop(["timestamp_field_7"],axis=1)

In [14]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98410 entries, 0 to 98409
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                98410 non-null  object
 1   order_id                 98410 non-null  object
 2   review_score             98410 non-null  int64 
 3   review_comment_title     98410 non-null  object
 4   review_comment_message   98410 non-null  object
 5   review_creation_date     98410 non-null  object
 6   review_answer_timestamp  98410 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


## 1.2 Gestion des doublons

In [58]:
df_reviews = df_reviews.drop_duplicates(['order_id','review_score','review_comment_title','review_comment_message','review_creation_date'])

In [33]:
df_reviews.shape

(98371, 7)

### analyse

In [19]:
df_reviews.drop_duplicates().shape

(98410, 7)

In [20]:
df_reviews.drop_duplicates(['order_id']).shape

(98167, 7)

In [21]:
df_reviews.drop_duplicates(['review_id']).shape

(98410, 7)

In [24]:
df_reviews[["order_id"]].value_counts().sort_values(ascending=False)

order_id                        
7b1219cc26d69c357880055b52d87960    2
b1e45a9d2437f4116c3ff5de92d2a527    2
76ac5a9b38c945eb910681d171e98c55    2
3ab974ac744bd19a111d5e19f0a90649    2
aa20064eef8808d62ed9ac5286ec0796    2
                                   ..
ff1ea7be87e1ca3d2ae5b77eaa6dab69    1
ff1f5f13541ce8f1a0755596df027a90    1
ff1f8f328e5a59ac754240b6dd31bcb4    1
ff207aa3493754ffcea819b8c0c0dbb5    1
fffe41c64501cc87c801fd61db3f6244    1
Name: count, Length: 98167, dtype: int64

In [31]:
df_reviews.loc[df_reviews["order_id"]=="aa20064eef8808d62ed9ac5286ec0796",:]

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
6796,662d2c533d56243557c12699eff9d5a3,aa20064eef8808d62ed9ac5286ec0796,5,,,2017-04-14 00:00:00,2017-04-16 19:41:32
73794,d83391dee674f4df74b3e72e31774030,aa20064eef8808d62ed9ac5286ec0796,5,,,2017-04-20 00:00:00,2017-04-23 11:14:56


## 1.3 Gestion des types de données

### 1.3.1 COnversion des typestamp

In [37]:
df_reviews.columns

Index(['review_id', 'order_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp'],
      dtype='object')

In [63]:
df_reviews.review_creation_date = pd.to_datetime(df_reviews['review_creation_date'], format= '%Y-%m-%d %H:%M:%S', errors="coerce")

In [64]:
df_reviews.review_answer_timestamp = pd.to_datetime(df_reviews['review_answer_timestamp'], format= '%Y-%m-%d %H:%M:%S', errors="coerce")

In [86]:
df_reviews.dtypes['review_creation_date']

dtype('<M8[ns]')

In [87]:
from numpy import dtype
df_reviews.dtypes['review_creation_date'] == dtype('<M8[ns]')


False

In [65]:
df_reviews.tail()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
98405,574ed12dd733e5fa530cfd4bbf39d7c9,2a8c23fee101d4d5662fa670396eb8da,5,,,2018-07-07,2018-07-14 17:18:30
98406,f3897127253a9592a73be9bdfdf4ed7a,22ec9f0669f784db00fa86d035cf8602,5,,,2017-12-09,2017-12-11 20:06:42
98407,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,5,,"Excelente mochila, entrega super rápida. Super...",2018-03-22,2018-03-23 09:10:43
98408,1adeb9d84d72fe4e337617733eb85149,7725825d039fc1f0ceb7635e3f7d9206,4,,,2018-07-01,2018-07-02 12:59:13
98409,efe49f1d6f951dd88b51e6ccd4cc548f,90531360ecb1eec2a1fbb265a0db0508,1,,"meu produto chegou e ja tenho que devolver, po...",2017-07-03,2017-07-03 21:01:49


In [66]:
df_reviews.shape

(98371, 7)

In [67]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98371 entries, 0 to 98409
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   review_id                98371 non-null  object        
 1   order_id                 98371 non-null  object        
 2   review_score             98371 non-null  int64         
 3   review_comment_title     98371 non-null  object        
 4   review_comment_message   98371 non-null  object        
 5   review_creation_date     98368 non-null  datetime64[ns]
 6   review_answer_timestamp  98344 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 6.0+ MB


### 1.3.2 Remplacement des dates manquantes

In [70]:
df_reviews = df_reviews.dropna(subset=['review_creation_date','review_answer_timestamp'])

In [71]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98344 entries, 27 to 98409
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   review_id                98344 non-null  object        
 1   order_id                 98344 non-null  object        
 2   review_score             98344 non-null  int64         
 3   review_comment_title     98344 non-null  object        
 4   review_comment_message   98344 non-null  object        
 5   review_creation_date     98344 non-null  datetime64[ns]
 6   review_answer_timestamp  98344 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 6.0+ MB


In [69]:
df_reviews.loc[df_reviews['review_answer_timestamp'].isna(),:]['review_score'].value_counts()

review_score
5    16
4     6
1     2
3     2
2     1
Name: count, dtype: int64

# 4 Consistance des données

In [73]:
df_reviews['review_score'].value_counts()

review_score
5    56866
4    18993
1    11277
3     8095
2     3113
Name: count, dtype: int64